In [3]:
!pip install ta

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for ta: started
  Running setup.py install for ta: finished with status 'done'


  DEPRECATION: ta is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from binance import Client
from datetime import datetime, timedelta
client = Client(api_key="jM9zEeTTzT7R7QGlNrh5tUpOc8l71hOxu7CewCL4M0NvbBp3ilgmEwMOOjJfCRVB",api_secret="iZxFmlz5bvROsHQPZTP5hlJtsxYjOu8wocOlLWzIDdhMzEQu7KyyHB2DseOYQ66s",testnet=True)

import os
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine, text
import pg8000.native
import pandas as pd
import numpy as np
import ta

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

connector = Connector()

def getconn() -> pg8000.native.Connection:
    conn: pg8000.native.Connection = connector.connect(
        "semiotic-pact-388619:us-central1:tradingbot",
        "pg8000",
        user="postgres",
        password="GuruSat.3",
        db="criptos"
    )
    return conn

connection = create_engine('postgresql+pg8000://',creator=getconn)

In [2]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1688276310048,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.10000000', 'locked': '0.10000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.10000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '3670.58050256', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [323]:
#day = datetime.utcnow() - timedelta(seconds=1000)
day = datetime.utcnow() - timedelta(1)


PAIR = 'BTCUSDT'

# with connection.connect() as conn:
#     df =conn.execute(text(""" SELECT * FROM "{PAIR}" WHERE "Tiempo" > '{day}' """.format(day=day,PAIR=PAIR))).mappings().all()
# df = pd.DataFrame(df)[['Tiempo', 'Precio']].set_index('Tiempo')
# df = df.sort_values(by='Tiempo')
# df

with connection.connect() as conn:
    df =conn.execute(text(""" SELECT * FROM "{PAIR}" """.format(PAIR=PAIR))).mappings().all()
df = pd.DataFrame(df)[['Tiempo', 'Precio']].set_index('Tiempo')
df = df.sort_values(by='Tiempo')
df

,Precio
Tiempo,
2023-06-14 09:12:03.975,25920.01
2023-06-14 09:12:05.109,25920.01
2023-06-14 09:12:06.210,25920.01
2023-06-14 09:12:06.929,25920.01
2023-06-14 09:12:07.559,25920.01
...,...
2023-07-03 00:24:55.793,30659.14
2023-07-03 00:25:00.496,30659.99
2023-07-03 00:25:01.646,30659.99


In [52]:
df[-90:-1]

,Precio
Tiempo,
2023-07-02 06:18:38.151,30485.46
2023-07-02 06:18:40.498,30485.46
2023-07-02 06:18:47.377,30485.46
2023-07-02 06:18:50.285,30485.47
2023-07-02 06:18:50.568,30485.46
...,...
2023-07-02 06:22:27.296,30498.11
2023-07-02 06:22:30.907,30496.32
2023-07-02 06:22:35.300,30494.09


In [197]:
def stochastic_oscillator(df,n=90,m=17,lags=16):
    df['%K'] = pd.Series()
    for i in range(n,len(df)):
        df['%K'][i-1] = ((df.Precio[i-1] - min(df.Precio[i-n:i]))/(max(df.Precio[i-n:i]) - min(df.Precio[i-n:i]))) * 100

    df['%D'] = df['%K'].rolling(m).mean()

    dfc = pd.DataFrame()
    dfv = pd.DataFrame()
    dfx = pd.DataFrame()
    df['Señal_stoch2'] = pd.Series()
    df['Señal_venta_stoch1'] = pd.Series()
    df['Señal_compra_stoch1'] = pd.Series()

    for i in range(1,lags):
        maskc = (df['%K'].shift(i) < 20) & (df['%D'].shift(i) < 20)
        maskv = (df['%K'].shift(i) > 80) & (df['%D'].shift(i) > 80)
        dfc = dfc.append(maskc,ignore_index=True)
        dfv = dfv.append(maskv,ignore_index=True)
    df['Señal_venta_stoch1'] = np.where(dfv.sum(axis=0),1,0)
    df['Señal_compra_stoch1'] = np.where(dfc.sum(axis=0),1,0)  

    mask =  df['%K'].between(20,80) & df['%D'].between(20,80)
    df['Señal_stoch2'] = np.where(dfx.append(mask,ignore_index=True).sum(axis=0),1,0)


def RSI(df,n=90):
    dfx = pd.DataFrame()
    df['Señal_venta_RSI'] = pd.Series()
    df['Señal_compra_RSI'] = pd.Series()
    df['RSI'] = ta.momentum.rsi(df.Precio,window=n)
    mask = df['RSI'] > 50
    df['Señal_compra_RSI'] = np.where(dfx.append(mask,ignore_index=True).sum(axis=0),1,0)
    mask = df['RSI'] < 50
    df['Señal_venta_RSI'] = np.where(dfx.append(mask,ignore_index=True).sum(axis=0),1,0)


def MACD(df):
    df['EMA12'] = df.Precio.ewm(span=12).mean()
    df['EMA26'] = df.Precio.ewm(span=26).mean()
    df['MACD'] = df.EMA12 - df.EMA26
    df['Signal'] = df.MACD.ewm(span=9).mean()
    df['Histogram'] = df.MACD - df.Signal
    df['Señal_venta_MACD'] = pd.Series()
    df['Señal_compra_MACD'] = pd.Series()
    for i in range(1,len(df)):
        if df.Histogram.iloc[i] > 0 and df.Histogram.iloc[i-1] < 0:
            df['Señal_compra_MACD'][i] = 1
        else:
            df['Señal_compra_MACD'][i] = 0

        if df.Histogram.iloc[i] < 0 and df.Histogram.iloc[i-1] > 0:
            df['Señal_venta_MACD'][i] = 1
        else:
            df['Señal_venta_MACD'][i] = 0


def Stochastich_RSI_MACD(df,n=90,m=17,lags=16):
    stochastic_oscillator(df,n=n,m=m,lags=lags)
    RSI(df,n=n)
    MACD(df)
    dfx = pd.DataFrame()
    df['Señal_Compra'] = pd.Series()
    df['Señal_Venta'] = pd.Series()
    mask_compra = (df['Señal_compra_stoch1'] == 1) & (df['Señal_stoch2'] == 1) & (df['Señal_compra_RSI'] == 1) & (df['Señal_compra_MACD'] == 1) 
    df['Señal_Compra'] = np.where(dfx.append(mask_compra,ignore_index=True).sum(axis=0),1,0)
    mask_venta = (df['Señal_venta_stoch1'] == 1) & (df['Señal_stoch2'] == 1) & (df['Señal_venta_RSI'] == 1) & (df['Señal_venta_MACD'] == 1) 
    df['Señal_Venta'] = np.where(dfx.append(mask_venta,ignore_index=True).sum(axis=0),1,0)

In [324]:
Stochastich_RSI_MACD(df,n=90,m=17,lags=90)

C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\1584977968.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['%K'] = pd.Series()
C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\1584977968.py:4: RuntimeWarning: invalid value encountered in double_scalars
  df['%K'][i-1] = ((df.Precio[i-1] - min(df.Precio[i-n:i]))/(max(df.Precio[i-n:i]) - min(df.Precio[i-n:i]))) * 100
C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\1584977968.py:11: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['Señal_stoch2'] = pd.Series()
C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\1584977968.py:12: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['Señal_

In [206]:
len(df[(df['Señal_Compra'] == 1)])

21

In [326]:
buy = df[(df['Señal_Compra'] == 1)]['Precio']
sell = df[(df['Señal_Venta'] == 1)]['Precio']

In [327]:
df.dropna(inplace=True)

In [328]:
# buy = buy[:-2]
buy

Tiempo
2023-06-14 09:44:54.175    25966.24
2023-06-14 09:48:08.523    25971.67
2023-06-14 09:54:45.919    25981.67
2023-06-14 10:39:42.599    25963.90
2023-06-14 10:39:57.724    25963.28
                             ...   
2023-07-02 21:25:30.890    30588.69
2023-07-02 21:38:07.360    30598.99
2023-07-02 22:03:38.459    30569.40
2023-07-02 22:04:54.652    30567.74
2023-07-03 00:08:18.752    30618.15
Name: Precio, Length: 1161, dtype: float64

In [329]:
# sell[sell.index > pd.to_datetime('2023-07-02 20:10:22.133')]
sell

Tiempo
2023-06-14 09:55:10.540    25979.45
2023-06-14 10:20:22.575    25969.31
2023-06-14 10:47:22.737    25957.62
2023-06-14 11:42:23.032    25961.39
2023-06-14 11:52:43.700    25971.05
                             ...   
2023-07-02 21:50:09.161    30574.99
2023-07-02 22:47:38.312    30582.47
2023-07-02 22:49:05.192    30574.76
2023-07-02 22:50:00.822    30593.12
2023-07-02 23:02:11.589    30638.95
Name: Precio, Length: 1087, dtype: float64

In [345]:
sellprices.index[j]
#buy[buy.index > buyprices.index[j]].index[0]

Timestamp('2023-07-02 23:02:11.589000')

In [346]:

sellprices = pd.Series(dtype=float)
buyprices = pd.Series(dtype=float)

while True:
    if sell.index[0] < buy.index[0]:
        sell = sell.drop(sell.index[0])
    elif buy.index[-1] > sell.index[-1]:
        buy = buy.drop(buy.index[-1])

    if buy.index[0] < sell.index[0] and buy.index[-1] < sell.index[-1]:
        break

p = 0
while True:

    if len(buyprices) == 0:
        buyprices = pd.concat([buyprices,buy[0:1]])
        sellprices = pd.concat([sellprices,sell[0:1]])
        j = 0
        continue

    if len(buy[buy.index > buyprices.index[j]]) == p:
                break

    if len(buyprices) > 0:
        p = 0
        x = len(buyprices)
        while True:    
            if x != len(buyprices):
                x = len(buyprices)
                p = 0

            if buy[buy.index > buyprices.index[j]].index[p] > sellprices.index[j]:
                buyprices = pd.concat([buyprices,buy[buy.index > buyprices.index[j]][p:p+1]])
                k = 0
                while True:
                    if buyprices.index[j+1] < sell[sell.index > sellprices.index[j]].index[k]:
                        sellprices = pd.concat([sellprices,sell[sell.index > sellprices.index[j]][k:k+1]])
                        j += 1
                        break
                    k += 1
            p += 1

            if len(buy[buy.index > buyprices.index[j]]) == p:
                break

C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\4169000829.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sellprices = pd.Series()
C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\4169000829.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  buyprices = pd.Series()
C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\4169000829.py:16: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  buyprices = buyprices.append(buy[0:1])
C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\4169000829.py:17: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sellprices = sellprices.append(sell[0:1])
C:

In [333]:
buy[buy.index > buyprices.index[j]].index[0]

Timestamp('2023-07-02 22:04:54.652000')

In [347]:
len(buyprices)

494

In [348]:
len(sellprices)

494

In [350]:
buyprices

2023-06-14 09:44:54.175    25966.24
2023-06-14 10:39:42.599    25963.90
2023-06-14 10:49:38.420    25958.67
2023-06-14 11:52:04.395    25973.63
2023-06-14 12:02:04.333    25979.11
                             ...   
2023-07-02 18:34:05.102    30467.27
2023-07-02 20:11:40.755    30547.98
2023-07-02 20:25:55.598    30551.84
2023-07-02 21:25:30.890    30588.69
2023-07-02 22:03:38.459    30569.40
Length: 494, dtype: float64

In [351]:
sellprices

2023-06-14 09:55:10.540    25979.45
2023-06-14 10:47:22.737    25957.62
2023-06-14 11:42:23.032    25961.39
2023-06-14 11:52:43.700    25971.05
2023-06-14 13:09:01.579    25999.79
                             ...   
2023-07-02 20:10:22.133    30539.80
2023-07-02 20:20:18.114    30547.76
2023-07-02 21:09:24.541    30581.39
2023-07-02 21:38:47.415    30593.82
2023-07-02 22:47:38.312    30582.47
Length: 494, dtype: float64

In [349]:
profits = []

for i in range(len(buyprices)):
    profit = buyprices[i] - sellprices[i]
    profits.append(profit*0.1)
profits

[-1.3209999999999127,
 0.6280000000002475,
 -0.2720000000001164,
 0.25800000000017465,
 -2.0680000000000294,
 2.814999999999782,
 3.714000000000306,
 5.427999999999884,
 -0.5010000000002037,
 3.4409999999999856,
 1.20099999999984,
 -7.588000000000102,
 5.827999999999884,
 2.077000000000044,
 34.91500000000015,
 -2.2409999999999854,
 4.272000000000117,
 32.77200000000012,
 1.0190000000002328,
 1.718999999999869,
 -0.925,
 4.523999999999797,
 1.9680000000000293,
 2.9239999999997965,
 -5.0919999999998256,
 -0.6079999999998109,
 3.495000000000073,
 -2.306000000000131,
 1.6649999999997818,
 1.387000000000262,
 0.5229999999999564,
 0.6639999999999419,
 0.11100000000005822,
 0.4489999999997963,
 -1.4939999999998692,
 -1.3949999999997091,
 1.306000000000131,
 -0.07800000000024739,
 -1.556999999999971,
 -0.07500000000000001,
 0.4919999999998254,
 -1.031999999999971,
 -0.08700000000026194,
 -0.10799999999981083,
 -9.140999999999986,
 0.1919999999998254,
 -0.9819999999999709,
 2.5509999999998403,

In [352]:
sum(profits)

-401.16899999999987

In [200]:
Stochastich_RSI_MACD(df,n=90,m=17,lags=60)

C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\1584977968.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['%K'] = pd.Series()
C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\1584977968.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['%K'][i-1] = ((df.Precio[i-1] - min(df.Precio[i-n:i]))/(max(df.Precio[i-n:i]) - min(df.Precio[i-n:i]))) * 100
C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\1584977968.py:11: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['Señal_stoch2'] = pd.Series()
C:\Users\tinma\AppData\Local\Temp\ipykernel_33304\1584977968.py:12: Fu

In [201]:
df.dropna(inplace=True)

In [202]:
df.describe()

,Precio,%K,%D,Señal_stoch2,Señal_venta_stoch1,Señal_compra_stoch1,Señal_venta_RSI,Señal_compra_RSI,RSI,EMA12,EMA26,MACD,Signal,Histogram,Señal_venta_MACD,Señal_compra_MACD,Señal_Compra,Señal_Venta
count,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000,30848.000000
mean,30532.636192,52.850935,52.855428,0.242285,0.611158,0.550311,0.479675,0.520325,50.912266,30532.634618,30532.633047,0.001571,0.001543,0.000028,0.022270,0.022303,0.000681,0.000616
std,38.535432,40.236718,37.591451,0.428473,0.487495,0.497470,0.499595,0.499595,17.023005,38.343859,38.079954,2.709639,2.575884,0.752041,0.147564,0.147669,0.026083,0.024811
min,30176.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.620786,30239.401805,30275.519583,-62.508757,-53.137327,-11.833034,0.000000,0.000000,0.000000,0.000000
25%,30506.500000,5.839416,14.401214,0.000000,0.000000,0.000000,0.000000,0.000000,40.422855,30506.799739,30506.875705,-0.677736,-0.663940,-0.184775,0.000000,0.000000,0.000000,0.000000
50%,30528.750000,58.287293,56.070167,0.000000,1.000000,1.000000,0.000000,1.000000,50.724700,30528.697109,30528.647178,0.007822,0.007757,-0.002782,0.000000,0.000000,0.000000,0.000000
75%,30562.530000,99.426108,91.435448,0.000000,1.000000,1.000000,1.000000,1.000000,61.088596,30562.627309,30563.112348,0.686593,0.676146,0.166850,0.000000,0.000000,0.000000,0.000000
max,30684.720000,100.000000,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,96.220085,30648.495962,30642.067959,23.242721,21.464757,13.024850,1.000000,1.000000,1.000000,1.000000


In [95]:
buyprices = df[df['Señal_Compra'] == 1][['Precio']]
sellprices = df[df['Señal_Venta'] == 1][['Precio']]

if buyprices.index[0] > sellprices.index[0]:
    sellprices = sellprices.drop(sellprices.index[0])
elif buyprices.index[-1] > sellprices.index[-1]:
    buyprices = buyprices.drop(buyprices.index[-1])

In [96]:
profits = []
for i in range(len(sellprices)):
    profit = buyprices.Precio[i] - sellprices.Precio[i]
    profits.append(profit)

In [97]:
sum(profits)

-389.2399999999143

### BACKTESTING

In [99]:
for i in range(1,len(df)):
    df1 = pd.DataFrame()
    df1 = df[0:i]
    if len(df1) > 30:
        Stochastich_RSI_MACD(df1)
        df1.dropna(inplace=True)
        buyprices = df[df['Señal_compra_MACD'] == 1][['Precio']]
        sellprices = df[df['Señal_venta_MACD'] == 1][['Precio']]

        if buyprices.index[0] > sellprices.index[0]:
            sellprices = sellprices.drop(sellprices.index[0])
        elif buyprices.index[-1] > sellprices.index[-1]:
            buyprices = buyprices.drop(buyprices.index[-1])

        
    

,Precio,%K,%D,RSI,EMA12,EMA26,MACD,Signal,Histogram,Señal_venta,Señal_compra
Tiempo,,,,,,,,,,,
2023-07-01 22:20:56.301,30592.01,100.0,66.666667,54.535355,30592.005921,30592.005599,0.000323,0.000163,0.000159,0.0,1.0
2023-07-01 22:21:00.177,30592.01,100.0,66.666667,54.535355,30592.006581,30592.006034,0.000548,0.000242,0.000306,0.0,0.0
2023-07-01 22:21:01.479,30592.01,100.0,100.000000,54.535355,30592.007130,30592.006416,0.000714,0.000338,0.000377,0.0,0.0
2023-07-01 22:21:05.490,30592.01,100.0,100.000000,54.535355,30592.007588,30592.006754,0.000834,0.000438,0.000396,0.0,0.0
2023-07-01 22:21:08.269,30592.01,100.0,100.000000,54.535355,30592.007970,30592.007054,0.000916,0.000535,0.000382,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2023-07-02 06:47:44.175,30496.01,100.0,66.666667,3.769173,30496.086699,30497.072390,-0.985690,-1.310574,0.324884,0.0,0.0
2023-07-02 06:47:46.696,30496.00,0.0,66.666667,3.739257,30496.073361,30496.992953,-0.919592,-1.232378,0.312785,0.0,0.0
2023-07-02 06:47:48.120,30496.01,100.0,66.666667,4.555075,30496.063613,30496.920142,-0.856529,-1.157208,0.300679,0.0,0.0
